In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

In [ ]:

def load_dataset(path="event_logs.csv"):
    if not os.path.exists(path):
        print("No dataset found. Generating synthetic event log data...")

        data = {
            "powershell_execs": np.random.randint(0, 10, 1000),
            "script_block_events": np.random.randint(0, 5, 1000),
            "wmi_calls": np.random.randint(0, 3, 1000),
            "scheduled_tasks": np.random.randint(0, 4, 1000),
            "registry_mods": np.random.randint(0, 5, 1000),
            "network_connections": np.random.randint(1, 20, 1000),
            "encoded_commands": np.random.randint(0, 2, 1000),
            "is_malware": np.random.randint(0, 2, 1000)
        }

        df = pd.DataFrame(data)
        df.to_csv(path, index=False)
    else:
        df = pd.read_csv(path)

    print(f"Dataset loaded with shape: {df.shape}")
    return df

In [ ]:
def preprocess(df):
    X = df.drop("is_malware", axis=1)
    y = df["is_malware"]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y, scaler


In [ ]:
def train_model(X_train, y_train):
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    return clf


In [ ]:
def evaluate(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Purples")
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

In [ ]:
def save(model, scaler):
    os.makedirs("models", exist_ok=True)
    joblib.dump(model, "models/fileless_rf_model.pkl")
    joblib.dump(scaler, "models/fileless_scaler.pkl")
    print("Saved model and scaler to 'models/'")

In [ ]:
if __name__ == "__main__":
    df = load_dataset()
    X, y, scaler = preprocess(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = train_model(X_train, y_train)
    evaluate(model, X_test, y_test)
    save(model, scaler)